In [1]:
import hnswlib
import numpy as np 
import struct
import heapq
import time
import sys
import os

In [2]:
def mmap_fvecs(fname):
    x = np.memmap(fname, dtype='int32', mode='r')
    d = x[0]
    return x.view('float32').reshape(-1, d + 1)[:, 1:]

def mmap_bvecs(fname):
    x = np.memmap(fname, dtype='uint8', mode='r')
    d = x[:4].view('int32')[0]
    return x.reshape(-1, d + 4)[:, 4:]

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    # Wenqi: Format of ground truth (for 10000 query vectors):
    #   1000(topK), [1000 ids]
    #   1000(topK), [1000 ids]
    #        ...     ...
    #   1000(topK), [1000 ids]
    # 10000 rows in total, 10000 * 1001 elements, 10000 * 1001 * 4 bytes
    return a.reshape(-1, d + 1)[:, 1:].copy()

def fvecs_read(fname):
    return ivecs_read(fname).view('float32')

In [3]:
dbname = 'SIFT1M'
index_path='../indexes/{}_index.bin'.format(dbname)
dim=128

if dbname.startswith('SIFT'):
    # SIFT1M to SIFT1000M
    dbsize = int(dbname[4:-1])
    xb = mmap_bvecs('/mnt/scratch/wenqi/Faiss_experiments/bigann/bigann_base.bvecs')
    xq = mmap_bvecs('/mnt/scratch/wenqi/Faiss_experiments/bigann/bigann_query.bvecs')
    gt = ivecs_read('/mnt/scratch/wenqi/Faiss_experiments/bigann/gnd/idx_%dM.ivecs' % dbsize)

    N_VEC = int(dbsize * 1000 * 1000)

    # trim xb to correct size
    xb = xb[:dbsize * 1000 * 1000]

    # Wenqi: load xq to main memory and reshape
    xq = xq.astype('float32').copy()
    xq = np.array(xq, dtype=np.float32)
    gt = np.array(gt, dtype=np.int32)

    print("Vector shapes:")
    print("Base vector xb: ", xb.shape)
    print("Query vector xq: ", xq.shape)
    print("Ground truth gt: ", gt.shape)
else:
    print('unknown dataset', dbname, file=sys.stderr)
    sys.exit(1)

Vector shapes:
Base vector xb:  (1000000, 128)
Query vector xq:  (10000, 128)
Ground truth gt:  (10000, 1000)


## Core HNSW load and search

In [4]:
def convertBytes(bytestring, dtype='int'):
    """
    convert bytes to a single element
    dtype = {int, long, float, double}
    struct: https://docs.python.org/3/library/struct.html
    """ 
    # int from bytes is much faster than struct.unpack
    if dtype =='int' or dtype == 'long': 
        return int.from_bytes(bytestring, byteorder='little', signed=False)
    elif dtype == 'float': 
        return struct.unpack('f', bytestring)[0]
    elif dtype == 'double': 
        return struct.unpack('d', bytestring)[0]
    else:
        raise ValueError 

# Wenqi: the fastest way to load a bytestring list is to use *** np.frombuffer ***
def convertBytesList(bytestring, dtype='int'):
    """
    Given a byte string, return the value list
    """
    result_list = []
    if dtype == 'int' or dtype == 'float':
        dsize = 4
    elif dtype == 'long' or dtype == 'double':
        dsize = 8
    else:
        raise ValueError 
        
    start_pointer = 0
    for i in range(len(bytestring) // dsize):
        result_list.append(convertBytes(
            bytestring[start_pointer: start_pointer + dsize], dtype=dtype))
        start_pointer += dsize
    return result_list

def calculateDist(query_data, db_vec):
    """
    HNSWLib returns L2 square distance, so do we
        both inputs are 1-d np array
    """
    # return l2 distance between two points
    return np.sum((query_data - db_vec) ** 2)


def merge_two_distance_list(list_A, list_B, k):
    """
    merge two lists by selecting the k pairs of the smallest distance
    input:
        both list has format [(dist, ID), (dist, ID), ...]
    return:
        a result list, with ascending distance (the first contains the largest distance)
    """
    
    results_heap = []
    for i in range(len(list_A)):
        dist, server_ID, vec_ID = list_A[i]
        heapq.heappush(results_heap, (-dist, server_ID, vec_ID))
    for i in range(len(list_B)):
        dist, server_ID, vec_ID = list_B[i]
        heapq.heappush(results_heap, (-dist, server_ID, vec_ID))

    while len(results_heap) > k:
        heapq.heappop(results_heap)

    results = []
    while len(results_heap) > 0:
        dist, server_ID, vec_ID = results_heap[0]
        results.append((-dist, server_ID, vec_ID))
        heapq.heappop(results_heap)
    results.reverse()
            
    return results

In [34]:
import struct 

        
class HNSW_index():
    
    """
    Returned result list always in the format of (dist, server_ID, vec_ID),
        in ascending distance order (the first result is the nearest neighbor)
    """
    
    def __init__(self, local_server_ID=0, dim=128):
        
        self.dim = dim
        self.local_server_ID = local_server_ID
        
        # Meta Info
        self.offsetLevel0_ = None
        self.max_elements_ = None
        self.cur_element_count = None
        self.size_data_per_element_ = None
        self.label_offset_ = None
        self.offsetData_ = None
        self.maxlevel_ = None
        self.enterpoint_node_ = None
        self.maxM_ = None
        self.maxM0_ = None
        self.M_ = None
        self.mult_ = None # the probability that a node is one a higher level
        self.ef_construction_ = None
        
        # ground layer, all with length of cur_element_count
        self.links_count_l0 = None # a list of link_count
        self.links_l0 = None # a list of links per vector
        self.data_l0 = None # a list of vectors
        label_l0 = None # a list of vector IDs
        
        # upper layers, all with length of cur_element_count
        self.element_levels_ = None # the level per vector
        self.links = None # the upper layer link info (link count + links)
        
        # remote nodes, order according to local ID (not label ID)
        #  remote_links: an 2-D array (cur_element_count, k), 
        #    each element is a tuple: (server_ID, vector_ID)
        self.remote_links_count = None
        self.remote_links = None
        
    def load_meta_info(self, index_bin):
        """
        index_bin = hnswlib index binary 
        
        HNSW save index order:
            https://github.com/WenqiJiang/hnswlib-eval/blob/master/hnswlib/hnswalg.h#L588-L616
        """
        self.offsetLevel0_ = int.from_bytes(index_bin[0:8], byteorder='little', signed=False)
        self.max_elements_ = int.from_bytes(index_bin[8:16], byteorder='little', signed=False)
        self.cur_element_count = int.from_bytes(index_bin[16:24], byteorder='little', signed=False)
        self.size_data_per_element_ = int.from_bytes(index_bin[24:32], byteorder='little', signed=False)
        self.label_offset_ = int.from_bytes(index_bin[32:40], byteorder='little', signed=False)
        self.offsetData_ = int.from_bytes(index_bin[40:48], byteorder='little', signed=False)
        self.maxlevel_ = int.from_bytes(index_bin[48:52], byteorder='little', signed=False)
        self.enterpoint_node_ = int.from_bytes(index_bin[52:56], byteorder='little', signed=False)
        self.maxM_ = int.from_bytes(index_bin[56:64], byteorder='little', signed=False)
        self.maxM0_ = int.from_bytes(index_bin[64:72], byteorder='little', signed=False)
        self.M_ = int.from_bytes(index_bin[72:80], byteorder='little', signed=False)
        self.mult_ = struct.unpack('d', index_bin[80:88])[0] # the probability that a node is one a higher level
        self.ef_construction_ = int.from_bytes(index_bin[88:96], byteorder='little', signed=False)
        

        print("offsetLevel0_", self.offsetLevel0_)
        print("max_elements_", self.max_elements_)
        print("cur_element_count", self.cur_element_count)
        print("size_data_per_element_", self.size_data_per_element_)
        print("label_offset_", self.label_offset_)
        print("offsetData_", self.offsetData_)
        print("maxlevel_", self.maxlevel_)
        print("enterpoint_node_", self.enterpoint_node_)
        print("maxM_", self.maxM_)
        print("maxM0_", self.maxM0_)
        print("M_", self.M_)
        print("mult_", self.mult_)
        print("ef_construction_", self.ef_construction_)
        
    
    def load_ground_layer(self, index_bin):
        """
        Get the ground layer vector ID, vectors, and links:
            links_count_l0: vec_num
            links_l0: maxM0_ * vec_num 
            data_l0: (dim, vec_num)
            label_l0: vec_num
        """
        
        # Layer 0 data 
        start_byte_pointer = 96
        delta = self.cur_element_count * self.size_data_per_element_
        data_level0 = index_bin[start_byte_pointer: start_byte_pointer + delta]
        
        size = len(data_level0)
        self.links_count_l0 = []
        self.links_l0 = np.zeros((self.cur_element_count, self.maxM0_), dtype=int)
        self.data_l0 = np.zeros((self.cur_element_count, self.dim))
        self.label_l0 = []

        data_l0_list = []
        
        assert len(data_level0) == self.size_data_per_element_ * self.cur_element_count
        
        size_link_count = 4
        size_links = self.maxM0_ * 4
        size_vectors = self.dim * 4
        size_label = 8
        
        assert self.size_data_per_element_ == \
            size_link_count + size_links + size_vectors + size_label
            
        for i in range(self.cur_element_count):
            # per ground layer node: (link_count (int), links (int array of len=maxM0_), 
            #    vector (float array of len=dim, vector ID (long)))
            
            addr_link_count = i * self.size_data_per_element_ 
            addr_links = addr_link_count + size_link_count
            addr_vectors = addr_links + size_links
            addr_label = addr_vectors + size_vectors
            
            tmp_bytes = data_level0[addr_link_count: addr_link_count + size_link_count]
            self.links_count_l0.append(convertBytes(tmp_bytes, dtype='int'))
        
            tmp_bytes = data_level0[addr_links: addr_links + size_links]
            self.links_l0[i] = np.frombuffer(tmp_bytes, dtype=np.int32)
            
            tmp_bytes = data_level0[addr_vectors: addr_vectors + size_vectors]
            self.data_l0[i] = np.frombuffer(tmp_bytes, dtype=np.float32)
            
            tmp_bytes = data_level0[addr_label: addr_label + size_label]
            self.label_l0.append(convertBytes(tmp_bytes, dtype='long'))


    def load_upper_layers(self, index_bin):
        """
        Get the upper layer info:
            element_levels_: the levels of each vector
            links: list of upper links
        """
        
        # meta + ground data
        start_byte_pointer = 96 + self.max_elements_ * self.size_data_per_element_
        
        # Upper layers
        size_links_per_element_ = self.maxM_ * 4 + 4
        self.element_levels_ = []
        self.links = []

        for i in range(self.cur_element_count):
            tmp_bytes = index_bin[start_byte_pointer:start_byte_pointer+4]
            linkListSize = convertBytes(tmp_bytes, dtype='int')
            start_byte_pointer += 4
            
            # if an element is only on ground layer, it has no links on upper layers at all
            if linkListSize == 0:
                self.element_levels_.append(0)
                self.links.append([])
            else:
                level = int(linkListSize / size_links_per_element_)
                self.element_levels_.append(level)
                tmp_bytes = index_bin[start_byte_pointer:start_byte_pointer+linkListSize]
                links_tmp = list(np.frombuffer(tmp_bytes, dtype=np.int32))
                start_byte_pointer += linkListSize
                self.links.append(links_tmp)

        assert start_byte_pointer == len(index_bin) # 6606296

    def save_as_FPGA_format(self, out_dir):
        """
        Save the data as FPGA format, must make sure `load_ground_layer` and `load_upper_layers` are already invoked.
        """
        # save meta data
        byte_array_meta = bytearray() 
        byte_array_meta += self.cur_element_count.to_bytes(4, byteorder='little', signed=False)
        byte_array_meta += self.maxlevel_.to_bytes(4, byteorder='little', signed=False)
        byte_array_meta += self.enterpoint_node_.to_bytes(4, byteorder='little', signed=False)
        byte_array_meta += self.maxM_.to_bytes(4, byteorder='little', signed=False)
        byte_array_meta += self.maxM0_.to_bytes(4, byteorder='little', signed=False)

        # save ground level links
        # format of each node:
        #   [64 B header = num_links (4B int) + 62 byte paddings] + N [64B actual links] + paddings (to 64 B)
        byte_array_ground_links = bytearray()
        for i in range(self.cur_element_count):
            link_count = self.links_count_l0[i]
            links = self.links_l0[i]
            byte_array_ground_links += int(link_count).to_bytes(4, byteorder='little', signed=False)
            byte_array_ground_links += b'\x00' * 60
            for j in range(self.maxM0_):
                if j < link_count:
                    byte_array_ground_links += int(links[j]).to_bytes(4, byteorder='little', signed=False)
                else:
                    byte_array_ground_links += b'\x00' * 4
            if len(byte_array_ground_links) % 64 != 0:
                byte_array_ground_links += b'\x00' * (64 - len(byte_array_ground_links) % 64)
            
        # save ground level labels
        # each node's physical position will be translates to its label
        byte_array_ground_labels = bytearray()
        for i in range(self.cur_element_count):
            label = self.label_l0[i]
            byte_array_ground_labels += int(label).to_bytes(4, byteorder='little', signed=False)
        assert len(byte_array_ground_labels) == self.cur_element_count * 4

        # save ground level vectors
        # format of each node:
        # [vector (4B float) + padding] + [visited (4B int, init as -1) + padding]
        byte_array_ground_vectors = bytearray()
        for i in range(self.cur_element_count):
            vector = self.data_l0[i]
            for j in range(self.dim):
                byte_array_ground_vectors += struct.pack('f', vector[j])
            if len(byte_array_ground_vectors) % 64 != 0:
                byte_array_ground_vectors += b'\x00' * (64 - len(byte_array_ground_vectors) % 64)
            byte_array_ground_vectors += b'\xff\xff\xff\xff' # -1 in int32
            byte_array_ground_vectors += b'\x00' * 60

        # save upper links, format:
        #   [num_links (4B int) + padding] + N [64B actual links] + paddings (to 64 B)
        byte_array_upper_links = bytearray()
        # save pointers to addresses of upper links, each is 4B int as a pointer to a byte address
        byte_array_upper_links_pointers = bytearray()
        for i in range(self.cur_element_count):
            levels = self.element_levels_[i] 
            pointer_addr = len(byte_array_upper_links) 
            byte_array_upper_links_pointers += pointer_addr.to_bytes(8, byteorder='little', signed=False)
            if levels != 0:
                links = self.links[i]
                for j in range(levels):
                    link_count = links[j * (1 + self.M_)]
                    byte_array_upper_links += int(link_count).to_bytes(4, byteorder='little', signed=False)
                    byte_array_upper_links += b'\x00' * 60
                    for k in range(self.M_):
                        byte_array_upper_links += int(links[j * (1 + self.M_) + 1 + k]).to_bytes(4, byteorder='little', signed=False)
                    if len(byte_array_upper_links) % 64 != 0:
                        byte_array_upper_links += b'\x00' * (64 - len(byte_array_upper_links) % 64)
        
        # save to files
        with open(os.path.join(out_dir, 'meta.bin'), 'wb') as f:
            f.write(byte_array_meta)
        with open(os.path.join(out_dir, 'ground_links.bin'), 'wb') as f:
            f.write(byte_array_ground_links)
        with open(os.path.join(out_dir, 'ground_labels.bin'), 'wb') as f:
            f.write(byte_array_ground_labels)
        with open(os.path.join(out_dir, 'ground_vectors.bin'), 'wb') as f:
            f.write(byte_array_ground_vectors)
        with open(os.path.join(out_dir, 'upper_links.bin'), 'wb') as f:
            f.write(byte_array_upper_links)
        with open(os.path.join(out_dir, 'upper_links_pointers.bin'), 'wb') as f:
            f.write(byte_array_upper_links_pointers)
        return byte_array_meta, byte_array_ground_links, byte_array_ground_labels, byte_array_ground_vectors, byte_array_upper_links, byte_array_upper_links_pointers

    def searchKnn(self, q_data, k, ef, debug=False):
        """
        result a list of (distance, vec_ID) in ascending distance
        """
        
        ep_node = self.enterpoint_node_
        num_elements = self.cur_element_count
        max_level = self.maxlevel_
        links_count_l0 = self.links_count_l0
        links_l0 = self.links_l0
        data_l0 = self.data_l0
        links = self.links
        label_l0 = self.label_l0
        dim = self.dim
        
        currObj = ep_node
        currVec = data_l0[currObj]
        curdist = calculateDist(q_data, currVec)
        
        search_path_local_ID = set()
        search_path_vec_ID = set()
        
        # search upper layers
        for level in reversed(range(1, max_level+1)):
            # if debug:
            #     print("")
            #     print("level: ", level)
            changed = True
            while changed:
                # if debug:
                #     print("current object: ", currObj, ", current distance: ", curdist)
                search_path_local_ID.add(currObj)
                changed = False
                ### Wenqi: here, assuming Node ID can be used to retrieve upper links (which is not true for indexes with ID starting from non-0)
                if (len(links[currObj])==0):
                    break
                else:
                    start_index = (level-1) * (1 + self.M_)
                    size = links[currObj][start_index]
                    # if debug:
                    #     print("size of neighbors: ", size) 
                    neighbors = links[currObj][(start_index+1):(start_index+(1 + self.M_))]
                    for i in range(size):
                        cand = neighbors[i]
                        currVec = data_l0[cand]
                        dist = calculateDist(q_data, currVec)
                        # if debug:
                        #     print("cand: ", cand, ", dist: ", dist)
                        if (dist < curdist):
                            curdist = dist
                            currObj = cand
                            changed = True
                    #         if debug:
                    #             print("changed")
                    # if debug:
                    #     print("one node finish")
                    #     print("")

        # search in ground layer
        if debug:
            print("level 0 entry point: ", currObj)
        visited_array = set() # default 0
        top_candidates = []
        candidate_set = []
        lowerBound = curdist 
        # By default heap queue is a min heap: https://docs.python.org/3/library/heapq.html
        # candidate_set = candidate list, min heap
        # top_candidates = dynamic list (potential results), max heap
        # compare min(candidate_set) vs max(top_candidates)
        heapq.heappush(top_candidates, (-curdist, currObj))
        heapq.heappush(candidate_set,(curdist, currObj))
        visited_array.add(currObj) 

        cnt_hops = 0
        max_cand_size = 0
        
        while len(candidate_set)!=0:
            current_node_pair = candidate_set[0]
            if ((current_node_pair[0] > lowerBound)):
                break
            else:
                cnt_hops += 1
                max_cand_size = max(max_cand_size, len(candidate_set))
            heapq.heappop(candidate_set)
            current_node_id = current_node_pair[1]
            search_path_local_ID.add(current_node_id)
            size = links_count_l0[current_node_id]
            if debug:
                print("current object: ", current_node_id)
            #     print("size of neighbors: ", size)
            for i in range(size):
                candidate_id = links_l0[current_node_id][i]
                if (candidate_id not in visited_array):
                    visited_array.add(candidate_id)
                    currVec = data_l0[candidate_id]
                    dist = calculateDist(q_data, currVec)
                    # if debug:
                    #     print("current object: ", candidate_id, ", current distance: ", dist, ", lowerBound: ", lowerBound)
                    if (len(top_candidates) < ef or lowerBound > dist):
                        # if debug:
                        #     print("added")
                        heapq.heappush(candidate_set, (dist, candidate_id))
                        heapq.heappush(top_candidates, (-dist, candidate_id))
                    if (len(top_candidates) > ef):
                        heapq.heappop(top_candidates)
                    if (len(top_candidates)!=0):
                        lowerBound = -top_candidates[0][0]
                else :
                    # if debug:
                    #     print("current object: ", candidate_id, ", visited already")
                    pass
            # if debug:
            #     print("one node finishes")
            #     print("")
        if debug:
            print("total hops: ", cnt_hops)
            print("max candidate set size: ", max_cand_size)

        while len(top_candidates) > k:
            heapq.heappop(top_candidates)

        result = []
        while len(top_candidates) > 0:
            candidate_pair = top_candidates[0]
            # Wenqi: here, replace the local candidate ID by real node ID, great!
            result.append([-candidate_pair[0], self.local_server_ID, label_l0[candidate_pair[1]]])
            heapq.heappop(top_candidates)
        result.reverse()
            
        for local_ID in search_path_local_ID:
            search_path_vec_ID.add(label_l0[local_ID])

        return result, search_path_local_ID, search_path_vec_ID
        

Links format 

bytestrings: 0 0 0 ... 

0 -> no upper layer
N -> N bytes for the following string, this is m x 4 x (1 + M), here M = 16 -> 

0 0 0 68 {string contents} 0 0 136 {string contents}

string contents: first element: valid edge number; rest: vector IDs

e.g.,
[
// first 1 + 16 elements
  10,
  
  1561,
  3999,
  4373,
  4213,
  178,
  6898,
  7020,
  7380,
  8454,
  8779,
  3498,
  3755,
  3999,
  4213,
  4373,
  4374,
  
// second 1 + 16 elements
  15,
  
  1191,
  1298,
  1311,
  1781,
  1930,
  2086,
  2598,
  2925,
  2936,
  3262,
  4374,
  4390,
  5441,
  5546,
  5607,
  0],

## Example Run

In [35]:
from pathlib import Path
index_path='../indexes/SIFT1M_index_M_32.bin'#.format(dbname)
index = Path(index_path).read_bytes()
len(index)

788325340

In [36]:
hnsw_index = HNSW_index(local_server_ID=0, dim=128)

### Example Output:


The parameters in the index header, stored in small endian
uint64_t offsetLevel0_; // 0:8
uint64_t max_elements_; // 8:16
uint64_t cur_element_count; // 16:24
uint64_t size_data_per_element_; // 24:32
uint64_t label_offset_; // 32:40
uint64_t offsetData_; // 40:48
uint32_t maxlevel_; // 48:52
uint32_t enterpoint_node_; // 52:56
uint64_t maxM_; // 56:64
uint64_t maxM0_; // 64:72
uint64_t M_; // 72:80
double mult_; // 80:88
uint64_t ef_construction_; // 88:96

Results I got from C++ on SIFT1M:

Index file size: 660564936
offsetLevel0_: 0
max_elements_: 1000000
cur_element_count: 1000000
size_data_per_element_: 652
label_offset_: 644
offsetData_: 132
maxlevel_: 5
enterpoint_node_: 572337
maxM_: 16
maxM0_: 32
M_: 16
mult_: 0.360674
ef_construction_: 128

size_data_per_element_ 652 = sizeVec (128 * 4) + maxM0_ * link_num (4) size_link_ID (32 * 4) + vec_ID (8) 
···

In [37]:
hnsw_index.load_meta_info(index)

offsetLevel0_ 0
max_elements_ 1000000
cur_element_count 1000000
size_data_per_element_ 780
label_offset_ 772
offsetData_ 260
maxlevel_ 4
enterpoint_node_ 572468
maxM_ 32
maxM0_ 64
M_ 32
mult_ 0.28853900817779266
ef_construction_ 128


In [38]:
hnsw_index.maxM0_

64

In [39]:
t0 = time.time()
HNSW_index.load_ground_layer(hnsw_index, index)
t1 = time.time()
print("time consumption: {:.2f} sec".format(t1 - t0))

time consumption: 3.22 sec


In [40]:
t0 = time.time()
HNSW_index.load_upper_layers(hnsw_index, index)
t1 = time.time()
print("time consumption: {:.2f} sec".format(t1 - t0))

time consumption: 0.85 sec


## Export to FPGA

In [41]:
out_dir = '../FPGA_indexes/SIFT1M_index_M_32'
if not os.path.exists(out_dir):
	os.makedirs(out_dir)
byte_array_meta, byte_array_ground_links, byte_array_ground_labels, byte_array_ground_vectors, byte_array_upper_links, byte_array_upper_links_pointers = \
	hnsw_index.save_as_FPGA_format(out_dir)

In [33]:
# Example 1
hw_id = 894
gt_id = 890
qid = 7380

# # Example 2
# hw_id = 448689
# gt_id = 448690
# qid = 9762

print("hw: ", hw_id, "vec: ", hnsw_index.data_l0[hw_id])
print("fpga format vec:", np.frombuffer(byte_array_ground_vectors[hw_id*(512+64):hw_id*(512+64)+512], dtype=np.float32))
print("gt: ", gt_id, gt[qid, :1], "vec: ", hnsw_index.data_l0[gt_id])
print("hw vec to query dist: ", calculateDist(hnsw_index.data_l0[hw_id], xq[qid]))
print("gt vec to query dist: ", calculateDist(hnsw_index.data_l0[gt_id], xq[qid]))
print("label of hw: ", hnsw_index.label_l0[hw_id])

hw:  894 vec:  [  0.   0.  57.  80.   0.   0.   0.   0.  23.  29. 121.  62.   0.   0.
   0.   3.  48.  32.  22.   1.   1.   0.   0.  10.   3.   1.   0.   0.
  15.   5.   2.   1.  21.  15. 150. 150.   0.   0.   0.   4. 150.  71.
 102.  44.   0.   0.   0.  35. 128.   6.   1.   0.   0.   0.   0.  44.
   2.   0.   0.   0.   2.   3.   4.   4.  46.   3.  30.  68.   2.   5.
  15.  75. 150.  53.   3.   1.   0.   0.   0.  50. 138.  41.   0.   0.
   0.   0.   0.   4.   2.   1.   0.   0.   0.   0.   2.   1.  11.   0.
   0.   0.   0.  13.  21.  57. 150.  71.   0.   0.   0.   0.   0.  26.
 115. 110.   0.   0.   0.   0.   0.   0.   3.  10.   7.   4.   0.   0.
   0.   0.]
fpga format vec: [  0.   0.  57.  80.   0.   0.   0.   0.  23.  29. 121.  62.   0.   0.
   0.   3.  48.  32.  22.   1.   1.   0.   0.  10.   3.   1.   0.   0.
  15.   5.   2.   1.  21.  15. 150. 150.   0.   0.   0.   4. 150.  71.
 102.  44.   0.   0.   0.  35. 128.   6.   1.   0.   0.   0.   0.  44.
   2.   0.   0.   0.   2.   3.   

In [32]:
result, search_path_local_ID, search_path_vec_ID = HNSW_index.searchKnn(hnsw_index, xq[7380], k=1, ef=64, debug=True)
print(result[0]) 

level 0 entry point:  918929
current object:  918929
current object:  811653
current object:  349580
current object:  894
current object:  63042
current object:  173046
current object:  63176
current object:  798394
current object:  200952
current object:  765327
current object:  913659
current object:  893074
current object:  284707
current object:  974340
current object:  843575
current object:  231237
current object:  843486
current object:  847884
current object:  659752
current object:  372729
current object:  165785
current object:  916366
current object:  287891
current object:  889510
current object:  40640
current object:  136029
current object:  662000
current object:  168015
current object:  752042
current object:  463486
current object:  759198
current object:  295089
current object:  513389
current object:  562740
current object:  366211
current object:  227784
current object:  198252
current object:  400320
current object:  510318
current object:  131254
current object:  

In [16]:
# pointer to byte address of upper links
byte_addr = int.from_bytes(byte_array_upper_links_pointers[hnsw_index.enterpoint_node_ * 8: (hnsw_index.enterpoint_node_ + 1) * 8], byteorder='little', signed=False)
print("pointer to byte address of upper links: ", byte_addr)

pointer to byte address of upper links:  3576960


In [26]:
byte_addr_num_links = None
if hnsw_index.M_ % 16 == 0:
	byte_addr_num_links = byte_addr + (hnsw_index.maxlevel_ - 2) * (64 + int(hnsw_index.M_ / 16) * 64)
else:
	byte_addr_num_links = byte_addr + (hnsw_index.maxlevel_ - 2) * (64 + (int(hnsw_index.M_ / 16) + 1) * 64)
print("pointer to byte address of num_links: ", byte_addr_num_links)
num_links = int.from_bytes(byte_array_upper_links[byte_addr_num_links: byte_addr_num_links + 4], byteorder='little', signed=False)
print("num_links: ", num_links)
for i in range(hnsw_index.maxM_):
	print("link id: ", int.from_bytes(byte_array_upper_links[byte_addr_num_links + 64 + i * 4: byte_addr_num_links + 64 + (i + 1) * 4], byteorder='little', signed=False))

pointer to byte address of num_links:  3577344
num_links:  27
link id:  97737
link id:  91217
link id:  174257
link id:  394819
link id:  412264
link id:  366149
link id:  193293
link id:  81663
link id:  449792
link id:  474872
link id:  512701
link id:  80780
link id:  9519
link id:  385788
link id:  337004
link id:  471933
link id:  490572
link id:  239996
link id:  599490
link id:  675533
link id:  807785
link id:  825690
link id:  860412
link id:  887504
link id:  890242
link id:  891339
link id:  983300
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0


In [23]:

print(len(byte_array_upper_links))
for i in range(100):
	print("link id: ", int.from_bytes(byte_array_upper_links[i * 4: (i + 1) * 4], byteorder='little', signed=False))

6291264
link id:  30
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  1350
link id:  54794
link id:  55272
link id:  37749
link id:  148651
link id:  34061
link id:  88287
link id:  156592
link id:  57932
link id:  60586
link id:  159531
link id:  186594
link id:  193644
link id:  200317
link id:  250063
link id:  252840
link id:  266150
link id:  380178
link id:  466526
link id:  479604
link id:  527600
link id:  561086
link id:  665617
link id:  729060
link id:  740709
link id:  808283
link id:  824896
link id:  852442
link id:  874612
link id:  995620
link id:  148651
link id:  153375
link id:  24
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  0
link id:  47762
link id:  126755
link id:  34005
link id:  68422
link i

In [28]:
hnsw_index.links[hnsw_index.enterpoint_node_][-2*hnsw_index.maxM_:-hnsw_index.maxM_]

[91217,
 174257,
 394819,
 412264,
 366149,
 193293,
 81663,
 449792,
 474872,
 512701,
 80780,
 9519,
 385788,
 337004,
 471933,
 490572,
 239996,
 599490,
 675533,
 807785,
 825690,
 860412,
 887504,
 890242,
 891339,
 983300,
 0,
 0,
 0,
 0,
 0,
 0]

## Search function

In [18]:
k=1
ef=8
nq = 10
I = np.zeros((nq, k), dtype=np.int64)
t0 = time.time()
for i in range(nq):
    result, search_path_local_ID, search_path_vec_ID = HNSW_index.searchKnn(hnsw_index, xq[i], k=k, ef=ef, debug=True) 
    I[i] = np.array([x[2] for x in result])
    if gt[i, 0] != I[i, 0]:
        print("Mismatch! query: ", i)
    print("ground truth: ", gt[i, 0], ", result: ", I[i, 0], "hnsw_dist: ", result[0][0])
t1 = time.time()
print("time consumption: {:.2f} sec".format(t1 - t0))

print(' ' * 4, '\t', 'R@1    R@10   R@100')
for rank in 1, 10, 100:
    if rank > k:
        break
    n_ok = (I[:nq, :rank] == gt[:nq, :1]).sum()
    print("{:.4f}".format(n_ok / float(nq)), end=' ')

level 0 entry point:  462014
current object:  462014
current object:  66855
current object:  583065
current object:  583350
current object:  504814
current object:  900184
current object:  205768
current object:  885651
current object:  504547
current object:  539223
current object:  219536
current object:  229024
current object:  344333
current object:  831624
total hops:  14
max candidate set size:  28
ground truth:  504814 , result:  504814 hnsw_dist:  61125.0
level 0 entry point:  284026
current object:  284026
current object:  154081
current object:  249619
current object:  142570
current object:  346239
current object:  588616
current object:  499950
current object:  528450
current object:  790327
current object:  789879
current object:  915931
current object:  693904
total hops:  12
max candidate set size:  57
ground truth:  588616 , result:  588616 hnsw_dist:  53409.0
level 0 entry point:  952976
current object:  952976
current object:  980826
current object:  724680
current ob

In [16]:
print(result)
print(gt[0,0])

[[61125.0, 0, 504814]]
504814


In [17]:
print(len(search_path_local_ID), search_path_local_ID) 

132 {504835, 582150, 233481, 899594, 500240, 582161, 813078, 847900, 928798, 928802, 582191, 133171, 572468, 335953, 301139, 422483, 539223, 900184, 454231, 240216, 944727, 191064, 318561, 487010, 582765, 328816, 582781, 945798, 831624, 803468, 123023, 294036, 372375, 504471, 543901, 602781, 229024, 90275, 504484, 647847, 504487, 36011, 810157, 583350, 504502, 462014, 746178, 371400, 296136, 596184, 581849, 312027, 581853, 696545, 504547, 7397, 276200, 593131, 504558, 7408, 296178, 601330, 846581, 851705, 582401, 467203, 983300, 893189, 344333, 370459, 504605, 76575, 505120, 278818, 454435, 209187, 504613, 331045, 66855, 274729, 329513, 372014, 941872, 370481, 941875, 260404, 272693, 627002, 428858, 99134, 815428, 283974, 720199, 426311, 296269, 429392, 581976, 183643, 296284, 486749, 366950, 126826, 996719, 421746, 61812, 648063, 576902, 360844, 219536, 885651, 292756, 583065, 344473, 294315, 504753, 100277, 295863, 567738, 23998, 504772, 205768, 682443, 8142, 544207, 233941, 582620, 

In [19]:
sorted_search_path_vec_ID = sorted(list(search_path_vec_ID)) 
sorted_search_path_local_ID = sorted(list(search_path_local_ID))

for i in range(len(sorted_search_path_local_ID)):
    if sorted_search_path_local_ID[i] != sorted_search_path_vec_ID[i]:
        print("i = {}\tsorted_search_path_local_ID = {}\tsorted_search_path_vec_ID = {}".format(
            i, sorted_search_path_local_ID[i], sorted_search_path_vec_ID[i]))

i = 2	sorted_search_path_local_ID = 8142	sorted_search_path_vec_ID = 8143
